In [1]:
import numpy as np
import tensorflow as tf
import os

from datetime import datetime
from sklearn.datasets import load_iris, make_classification
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
iris = load_iris()
X = iris.data[:, (2, 3)]
y = (iris.target == 0).astype(np.int)

### Perceptron

In [4]:
per_clf = Perceptron(random_state=42)
per_clf.fit(X, y)

c:\programdata\miniconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      max_iter=None, n_iter=None, n_jobs=1, penalty=None, random_state=42,
      shuffle=True, tol=None, verbose=0, warm_start=False)

In [6]:
y_pred = per_clf.predict([[2, 0.5]])

### DNN (MLP) High Level

In [47]:
X_10, y_10 = make_classification(n_samples=1000, n_classes=10, n_features=4, n_informative=4, n_redundant=0, n_repeated=0, n_clusters_per_class=1, random_state=42)
X_10 = X_10.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X_10, y_10)

In [48]:
X_10.dtype

dtype('float32')

In [49]:
feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)

In [50]:
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300, 100], n_classes=10, feature_columns=feature_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000001AD9B668>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\POKROV~1\\AppData\\Local\\Temp\\tmp4ak2r_zn'}


In [52]:
dnn_clf_sk = tf.contrib.learn.SKCompat(dnn_clf)
dnn_clf_sk.fit(X_train, y_train, batch_size=50, steps=4000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\POKROV~1\AppData\Local\Temp\tmp4ak2r_zn\model.ckpt-400
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 400 into C:\Users\POKROV~1\AppData\Local\Temp\tmp4ak2r_zn\model.ckpt.
INFO:tensorflow:loss = 0.5987427, step = 401
INFO:tensorflow:global_step/sec: 763.36
INFO:tensorflow:loss = 0.4615118, step = 501 (0.131 sec)
INFO:tensorflow:global_step/sec: 999.999
INFO:tensorflow:loss = 0.34938648, step = 601 (0.110 sec)
INFO:tensorflow:global_step/sec: 806.453
INFO:tensorflow:loss = 0.5087976, step = 701 (0.114 sec)
INFO:tensorflow:global_step/sec: 1075.26
INFO:tensorflow:loss = 0.55055463, step = 801 (0.103 sec)
INFO:tensorflow:global_step/sec: 813.01
INFO:tensorflow:loss = 0.46410385, step = 901 (0.113 sec)
INFO:tensorflow:global_step/sec: 961.538
INFO:tensorflow:loss = 0.39882973, step = 10

SKCompat()

### DNN (MLP) Low Level

In [2]:
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10
learning_rate = 0.01

tf.reset_default_graph()

X = tf.placeholder(tf.float32, (None, n_inputs), "X")
y = tf.placeholder(tf.int32, (None), "y")

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
with tf.name_scope("dnn"):
#     hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
#     hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
#     logits = neuron_layer(hidden2, n_outputs, name="outputs")
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float16))
    acc_summary = tf.summary.scalar('accuracy', accuracy)
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [3]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid = X_train[:5000]
y_valid = y_train[:5000]

In [4]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield (X_batch, y_batch)
        
def get_logdir():
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_logdir = "tf_logs"
    return "{}/run-{}/".format(root_logdir, now)

In [5]:
n_epochs = 10
batch_size = 50

checkpoint_path = "/tmp/mnist_model.ckpt"
checkpoint_epoch_path = checkpoint_path + ".epoch"
final_model_path = "tmp/mnist_model_final.ckpt"

file_writer = tf.summary.FileWriter(get_logdir(), tf.get_default_graph())

with tf.Session() as sess:
    if os.path.isfile(checkpoint_epoch_path):
        with open(checkpoint_epoch_path, "rb") as f:
            start_epoch = int(f.read())
        print("Training was interrupted. Continuing at epoch", start_epoch)
        saver.restore(sess, checkpoint_path)
    else:
        start_epoch = 0
        sess.run(init)
    
    for epoch in range(start_epoch, n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        
        acc_test, summary_str = sess.run([accuracy, acc_summary], feed_dict={X: X_test, y: y_test})
        
        file_writer.add_summary(summary_str, epoch)
        save_path = saver.save(sess, checkpoint_path)
        
        with open(checkpoint_epoch_path, "wb") as f:
            f.write(b"%d" % (epoch + 1))
        
        print(epoch, "Accuracy Train:", acc_train, "Test", acc_test)
        
    save_path = saver.save(sess, final_model_path)
    file_writer.close()
    os.remove(checkpoint_epoch_path)

0 Accuracy Train: 0.94 Test 0.2048
1 Accuracy Train: 0.94 Test 0.2048
2 Accuracy Train: 0.94 Test 0.2048
3 Accuracy Train: 0.88 Test 0.2048
4 Accuracy Train: 0.98 Test 0.2048
5 Accuracy Train: 0.9 Test 0.2048
6 Accuracy Train: 0.98 Test 0.2048
7 Accuracy Train: 0.94 Test 0.2048
8 Accuracy Train: 1.0 Test 0.2048
9 Accuracy Train: 1.0 Test 0.2048


In [6]:
with tf.Session() as sess:
    saver.restore(sess, final_model_path)
    Z = logits.eval(feed_dict={X: X_valid[:10]})
    y_pred = np.argmax(Z, axis=1)
    print(y_pred)

INFO:tensorflow:Restoring parameters from tmp/mnist_model_final.ckpt
[5 0 4 1 9 2 1 3 1 4]
